[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/transformers/HuggingFace%20in%20Spark%20NLP%20-%20BERT%20Sentence.ipynb)

## Import BERT models for Sentence Embeddings from HuggingFace 🤗 into Spark NLP 🚀 

Let's keep in mind a few things before we start 😊 

- This feature is only in `Spark NLP 3.1.x` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import models for BERT from HuggingFace but they have to be compatible with `TensorFlow` and they have to be in `Fill Mask` category. Meaning, you cannot use BERT models trained/fine-tuned on a specific task such as token/sequence classification.

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.4.1` version and Transformers on `4.6.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [ ]:
!pip install -q transformers==4.6.1 tensorflow==2.4.1

     |████████████████████████████████| 2.3MB 7.6MB/s 
     |████████████████████████████████| 394.3MB 40kB/s 
     |████████████████████████████████| 901kB 9.4MB/s 
     |████████████████████████████████| 3.3MB 35.2MB/s 
     |████████████████████████████████| 2.9MB 24.4MB/s 
     |████████████████████████████████| 3.8MB 30.9MB/s 
     |████████████████████████████████| 471kB 26.3MB/s 


- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [bert-base-cased](https://huggingface.co/bert-base-cased) model from HuggingFace as an example
- In addition to `TFBertModel` we also need to save the `BertTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [ ]:
from transformers import TFBertModel, BertTokenizer 

MODEL_NAME = 'bert-base-cased'

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME).save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))
model = TFBertModel.from_pretrained(MODEL_NAME).save_pretrained("./{}".format(MODEL_NAME), saved_model=True)

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

INFO:tensorflow:Assets written to: ./bert-base-cased/saved_model/1/assets


INFO:tensorflow:Assets written to: ./bert-base-cased/saved_model/1/assets


Let's have a look inside these two directories and see what we are dealing with:

In [ ]:
!ls -l {MODEL_NAME}

total 423352
-rw-r--r-- 1 root root       597 Jun  6 08:45 config.json
drwxr-xr-x 3 root root      4096 Jun  6 08:45 saved_model
-rw-r--r-- 1 root root 433503000 Jun  6 08:45 tf_model.h5


In [ ]:
!ls -l {MODEL_NAME}/saved_model/1

total 7736
drwxr-xr-x 2 root root    4096 Jun  6 08:45 assets
-rw-r--r-- 1 root root 7910827 Jun  6 08:45 saved_model.pb
drwxr-xr-x 2 root root    4096 Jun  6 08:45 variables


In [ ]:
!ls -l {MODEL_NAME}_tokenizer

total 220
-rw-r--r-- 1 root root    112 Jun  6 08:44 special_tokens_map.json
-rw-r--r-- 1 root root    521 Jun  6 08:44 tokenizer_config.json
-rw-r--r-- 1 root root 213450 Jun  6 08:44 vocab.txt


- as you can see, we need the SavedModel from `saved_model/1/` path
- we also be needing `vocab.txt` from the tokenizer
- all we need is to just copy the `vocab.txt` to `saved_model/1/assets` which Spark NLP will look for

In [ ]:
!cp {MODEL_NAME}_tokenizer/vocab.txt {MODEL_NAME}/saved_model/1/assets

## Import and Save BERT in Spark NLP for Sentence/Document embeddings
### `BertSentenceEmbeddings` is the equivalent annotator for this task
The output of this annotator can be used in multi-class/multi-label text classifications (`ClassifierDL`, `SentimentDL`, and `MultiClassifierDL`) 


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `BertSentenceEmbeddings` which allows us to load TensorFlow model in SavedModel format
- Unlike `BertEmbeddings` which uses `last_hidden_state` with (-1, -1, DIMENSION) shape, `BertSentenceEmbeddings` will use `pooler_output` with (-1, DIMENSION) shape for Sentence/Document embeddings. It will generate 1 vector for the entire sentence/document
- Most params can be set later when you are loading this model in `BertSentenceEmbeddings` in runtime, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- `setStorageRef` is very important. When you are training a task like NER or any Text Classification, we use this reference to bound the trained model to this specific embeddings so you won't load a different embeddings by mistake and see terrible results 😊
- It's up to you what you put in `setStorageRef` but it cannot be changed later on. We usually use the name of the model to be clear, but you can get creative if you want! 
- The `dimension` param is is purely cosmetic and won't change anything. It's mostly for you to know later via `.getDimension` what is the dimension of your model. So set this accordingly.


In [ ]:
from sparknlp.annotator import *

sent_bert = BertSentenceEmbeddings.loadSavedModel(
     '{}/saved_model/1'.format(MODEL_NAME),
     spark
 )\
 .setInputCols("sentence")\
 .setOutputCol("bert_sentence")\
 .setCaseSensitive(True)\
 .setDimension(768)\
 .setStorageRef('sent_bert_base_cased') 

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
sent_bert.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your BERT model for Sentence/Document embeddings from HuggingFace 🤗  loaded and saved by Spark NLP 🚀 

In [ ]:
! ls -l {MODEL_NAME}_spark_nlp

total 393292
-rw-r--r-- 1 root root 402718696 May 30 13:04 bert_tensorflow
drwxr-xr-x 4 root root      4096 May 30 13:02 fields
drwxr-xr-x 2 root root      4096 May 30 13:02 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny BERT model 😊 

In [ ]:
sent_bert_loaded = BertSentenceEmbeddings.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols("sentence")\
  .setOutputCol("bert_sentence")\
  .setCaseSensitive(True)

In [ ]:
sent_bert_loaded.getStorageRef()

'sent_bert_base_cased'

That's it! You can now go wild and use hundreds of BERT models from HuggingFace 🤗 in Spark NLP 🚀 
